In [1]:
import cv2
import os

import numpy as np
from numpy import prod
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, concatenate, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "3"  # Set the GPU 2 to use

In [2]:
def preprocess(location, save_path, file_name):
    base = tf.keras.applications.MobileNetV3Small(input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet', include_top=False)
    features = np.empty((0, 28224))
    labels = []
    base_model = tf.keras.Sequential([
      base,
      Flatten()
    ])
    base_model.summary()
    base_model.trainable = False
    index = 0
    label = location[-1]
    for lb, i in enumerate(os.listdir(location)): #location = new_datasets/train/1
        filedir = os.path.join(location, i)
        for j in os.listdir(filedir):
            split_video = os.path.join(filedir, j)
            frames = np.array(list(map(lambda x: os.path.join(split_video, x), os.listdir(split_video))))
            frames = sorted(frames, key=lambda x: int(x.split("/")[-1].strip(".png")))

            if len(frames) > 3000:
                print("frames over : ", len(frames))
                n = len(frames) - 3000
                n = n//2
                frames = frames[n:-n]
                
            cutline = len(frames) % LSTM_INTERVAL
            if cutline:
                frames = frames[:-cutline]
            
            if os.path.isfile(save_path+'/'+file_name+'_'+str(index)+'.npy'):
                index+=len(frames)//45
                continue
            labels.append([label]*(len(frames)))
            frames = np.array(list(map(cv2.imread, frames)))
            frames = np.array(list(map(lambda x: cv2.resize(x, (IMG_SIZE, IMG_SIZE)), frames)))

            if len(frames) <45:
                continue

            feature_list = base_model.predict(frames)
            for ind in range(0, len(feature_list), 45):

                np.save(save_path+'/'+file_name+'_'+str(index)+'.npy',feature_list[ind:ind+45])
                index+=1
        print("doing", index)
        if index > 6000:
            break
    print("done")

In [3]:
LSTM_INTERVAL = 45
IMG_SIZE = 224

In [9]:
location = "new_datasets/train/2"
save_path = "/home/nplab/Jueon/LSTM/final_datasets/train/2"
file_name = 'features_train_normal'
preprocess(location, save_path, file_name)

224


2022-05-16 11:49:58.500061: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-16 11:49:58.937032: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10157 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:68:00.0, compute capability: 6.1


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 28224)             0         
                                                                 
Total params: 939,120
Trainable params: 927,008
Non-trainable params: 12,112
_________________________________________________________________
frames over :  9065
doing 66
frames over :  7777
doing 132
frames over :  6714
doing 198
frames over :  6911
doing 264
frames over :  9041
doing 330
frames over :  9389
doing 396
frames over :  8750
doing 462
frames over :  9184
doing 528
frames over :  9038
doing 594
frames over :  5460
doing 660
frames over :  4037
doing 726
frames over :  5340
doing 792
frames over :  5

2022-05-16 11:54:20.285272: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2022-05-16 11:54:20.533384: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


doing 3102
frames over :  8798
doing 3168
frames over :  8777
doing 3234
frames over :  7490
doing 3300
frames over :  9203
doing 3366
frames over :  8329
doing 3432
frames over :  6843
doing 3498
frames over :  6686
doing 3564
frames over :  9172
doing 3630
frames over :  7738
doing 3696
frames over :  4880
doing 3762
frames over :  6720
doing 3828
frames over :  9372
doing 3894
frames over :  6613
doing 3960
frames over :  7074
doing 4026
frames over :  9183
doing 4092
doing 4155
frames over :  7233
doing 4221
frames over :  6376
doing 4287
frames over :  5918
doing 4353
frames over :  8685
doing 4419
frames over :  9255
doing 4485
frames over :  6624
doing 4551
frames over :  4747
doing 4617
frames over :  9171
doing 4683
frames over :  7341
doing 4749
frames over :  6625
doing 4815
frames over :  8620
doing 4881
frames over :  8026
doing 4947
frames over :  8936
doing 5013
frames over :  7490
doing 5079
frames over :  6589
doing 5145
frames over :  6742
doing 5211
frames over :  78